In [1]:
import pandas as pd 
from urllib.request import urlopen
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time
import pprint

In [2]:
session_key_list = pd.read_csv('session_keys.csv')

In [3]:

for i in session_key_list['session_key']:
    response = urlopen(f'https://api.openf1.org/v1/race_control?session_key={i}')
    data = json.loads(response.read().decode('utf-8'))
    if i == 7953:
        df_race_control = pd.DataFrame(data)
    else:
        hold = pd.DataFrame(data)
        df_race_control = pd.concat([df_race_control,hold])
        time.sleep(0.5)
    print(f'Session {i} extracted.')
print('Done')

Session 7953 extracted.
Session 7779 extracted.
Session 7787 extracted.
Session 9070 extracted.
Session 9078 extracted.
Session 9094 extracted.


/var/folders/xc/y6vw2n510h1fmnwd2wj1zqdc0000gn/T/ipykernel_97075/2412331064.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_race_control = pd.concat([df_race_control,hold])


Session 9102 extracted.
Session 9110 extracted.
Session 9118 extracted.
Session 9126 extracted.
Session 9133 extracted.
Session 9141 extracted.
Session 9149 extracted.
Session 9157 extracted.
Session 9165 extracted.
Session 9173 extracted.
Session 9221 extracted.
Session 9213 extracted.
Session 9181 extracted.
Session 9205 extracted.
Session 9189 extracted.
Session 9197 extracted.
Session 9472 extracted.
Session 9480 extracted.
Session 9488 extracted.
Session 9496 extracted.
Session 9673 extracted.
Session 9507 extracted.
Session 9515 extracted.
Session 9523 extracted.
Session 9531 extracted.
Session 9539 extracted.
Session 9550 extracted.
Session 9558 extracted.
Session 9566 extracted.
Session 9574 extracted.
Session 9582 extracted.
Session 9590 extracted.
Session 9598 extracted.
Session 9606 extracted.
Session 9617 extracted.
Session 9625 extracted.
Session 9636 extracted.
Session 9644 extracted.
Session 9655 extracted.
Session 9662 extracted.
Session 9693 extracted.
Session 9998 ext

In [4]:
df_race_control

,meeting_key,session_key,date,driver_number,lap_number,category,flag,scope,sector,message
0,1141,7953,2023-03-05T14:04:34+00:00,NaN,1.0,Other,None,None,NaN,PINK HEAD PADDING MATERIAL MUST BE USED
1,1141,7953,2023-03-05T14:20:00+00:00,NaN,1.0,Flag,GREEN,Track,NaN,GREEN LIGHT - PIT EXIT OPEN
2,1141,7953,2023-03-05T14:23:00+00:00,NaN,1.0,Flag,YELLOW,Sector,10.0,YELLOW IN TRACK SECTOR 10
3,1141,7953,2023-03-05T14:23:04+00:00,NaN,1.0,Flag,CLEAR,Sector,10.0,CLEAR IN TRACK SECTOR 10
4,1141,7953,2023-03-05T14:30:00+00:00,NaN,1.0,Other,None,None,NaN,PIT EXIT CLOSED
...,...,...,...,...,...,...,...,...,...,...
56,1274,9858,2025-11-23T05:04:26+00:00,NaN,37.0,Other,None,None,NaN,FIA STEWARDS: PENALTY SERVED - 5 SECOND TIME P...
57,1274,9858,2025-11-23T05:17:22+00:00,43.0,45.0,Flag,BLUE,Driver,NaN,WAVED BLUE FLAG FOR CAR 43 (COL) TIMED AT 21:1...
58,1274,9858,2025-11-23T05:22:21+00:00,30.0,49.0,Flag,BLUE,Driver,NaN,WAVED BLUE FLAG FOR CAR 30 (LAW) TIMED AT 21:2...
59,1274,9858,2025-11-23T05:25:17+00:00,NaN,50.0,Flag,CHEQUERED,Track,NaN,CHEQUERED FLAG


In [5]:
config = dotenv_values()

api_key = config['x-rapidapi-key']
pg_user = config['POSTGRES_USER']
pg_pass = config['POSTGRES_PASS']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [ ]:
dtype_dict = {
    'meeting_key': types.Integer,
    'session_key': types.Integer,
    'date': types.DateTime(timezone=True),
    'driver_number': types.Integer,   
    'lap_number': types.Float,         
    'category': types.String,
    'flag': types.String,             
    'scope': types.String,            
    'sector': types.Float,            
    'message': types.Text             
}

In [7]:

df_race_control.to_sql(name = 'raw_race_control', 
                       con = engine, 
                       schema = pg_schema, 
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

940